In [6]:
import pandas as pd
import seaborn
import matplotlib.pyplot as plt

import re
from collections import Counter

In [7]:
DATA_FOLDER = '../data/raw/train_dataset_train.csv'
df = pd.read_csv(DATA_FOLDER, sep=';')

In [8]:
df

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу
...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления


In [9]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_84740/2340421553.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [10]:
health_df = df[df['Группа тем']=='Здравоохранение/Медицина']

In [11]:
health_df

,Исполнитель,Группа тем,Текст инцидента,Тема


In [12]:
df['Тема'].values

array(['★ ямы во дворах', 'оказание гос. соц. помощи',
       'дети и многодетные семьи', ...,
       'благоустройство придомовых территорий',
       'ненадлежащее качество или отсутствие отопления',
       'оказание гос. соц. помощи'], dtype=object)

In [13]:
df_queue = df[df['Тема'].str.contains('очередь')]

In [14]:
df_queue['Группа тем'].unique()

array(['здравоохранение/медицина'], dtype=object)

In [15]:
df_children = df[df['Текст инцидента'].str.contains('дети')]
df_children['Группа тем'].unique()

array(['образование', 'благоустройство', 'мфц "мои документы"',
       'здравоохранение/медицина', 'дороги',
       'социальное обслуживание и защита', 'жкх',
       'общественный транспорт', 'физическая культура и спорт',
       'мусор/свалки/тко', 'коронавирус', 'безопасность',
       'строительство и архитектура', 'мобилизация', 'спецпроекты',
       'роспотребнадзор', 'электроснабжение',
       'памятники и объекты культурного наследия'], dtype=object)

In [62]:
def word_in_topics(df:pd.DataFrame, string: str):
    """
    возвращает список тем, в которых встречается слово
    """
    df_tmp = df[df['Текст инцидента'].str.contains(string)]
    print(df_tmp['Группа тем'].unique())
    print('------------------------------------------------------------------------------------------------------')
    print(df_tmp.index)
    if len (df_tmp.index) > 0:
        for i in df_tmp.index:
            print(df.iloc[i,2])
    return df_tmp['Группа тем'].unique()

In [63]:
word_in_topics(df, 'мусор')

['мусор/свалки/тко' 'благоустройство' 'жкх' 'общественный транспорт'
 'здравоохранение/медицина' 'дороги' 'безопасность' 'экология'
 'связь и телевидение' 'социальное обслуживание и защита'
 'строительство и архитектура' 'газ и топливо' 'торговля'
 'роспотребнадзор' 'образование']
------------------------------------------------------------------------------------------------------
Index([   26,    39,    40,    41,    44,    53,    74,   124,   125,   185,
       ...
       22938, 22963, 22975, 23025, 23050, 23051, 23056, 23082, 23108, 23118],
      dtype='int64', length=1217)
'😡 кто отвечает за вывоз мусора на верхней губахе?<br>поставили один бачок, для галочки, а вывозить его не надо? поставьте два, три, четыре бака…, или этот уберите совсем! устроили свалку, собаки разносят по всему посёлку. гости города приезжают на висячий мост, посмотреть самый длинный мост в пермском крае, и что они видят...? помойку…
'сегодня .дамба по пр .победы завалена мусором из урн .неужели вопрос уборки

array(['мусор/свалки/тко', 'благоустройство', 'жкх',
       'общественный транспорт', 'здравоохранение/медицина', 'дороги',
       'безопасность', 'экология', 'связь и телевидение',
       'социальное обслуживание и защита', 'строительство и архитектура',
       'газ и топливо', 'торговля', 'роспотребнадзор', 'образование'],
      dtype=object)

In [64]:
word2topcs = {}

In [65]:
word2topcs['мусор'] = word_in_topics(df, 'мусор')
word2topcs['дети'] = word_in_topics(df, 'дети')
word2topcs['очередь'] = word_in_topics(df, 'очередь')
word2topcs['асфальт'] = word_in_topics(df, 'асфальт')
word2topcs['срочно'] = word_in_topics(df, 'срочно')
word2topcs['помогите'] = word_in_topics(df, 'помогите')
word2topcs['плохо'] = word_in_topics(df, 'плохо')
word2topcs['управляющая'] = word_in_topics(df, 'управляющая')
word2topcs['вопрос'] = word_in_topics(df, 'вопрос')




IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [66]:
word2topcs

{'мусор': array(['мусор/свалки/тко', 'благоустройство', 'жкх',
        'общественный транспорт', 'здравоохранение/медицина', 'дороги',
        'безопасность', 'экология', 'связь и телевидение',
        'социальное обслуживание и защита', 'строительство и архитектура',
        'газ и топливо', 'торговля', 'роспотребнадзор', 'образование'],
       dtype=object),
 'дети': array(['образование', 'благоустройство', 'мфц "мои документы"',
        'здравоохранение/медицина', 'дороги',
        'социальное обслуживание и защита', 'жкх',
        'общественный транспорт', 'физическая культура и спорт',
        'мусор/свалки/тко', 'коронавирус', 'безопасность',
        'строительство и архитектура', 'мобилизация', 'спецпроекты',
        'роспотребнадзор', 'электроснабжение',
        'памятники и объекты культурного наследия'], dtype=object),
 'очередь': array(['здравоохранение/медицина', 'социальное обслуживание и защита',
        'образование', 'жкх', 'благоустройство', 'общественный транспорт',
 

In [67]:
all_text = ' '.join(df['Текст инцидента'])

# Разделение строки на отдельные слова
words = re.findall(r'\w+', all_text.lower())

# Подсчет частоты каждого слова
word_freq = Counter(words)

# Преобразование результатов подсчета в DataFrame
word_freq_df = pd.DataFrame(list(word_freq.items()), columns=['Слово', 'Частота'])

# Сохранение DataFrame в CSV
word_freq_df.to_csv('word_freq.csv', index=False)

In [68]:
word_group_mapping = {}

# Проход по каждой строке датасета
for index, row in df.iterrows():
    text = row['Текст инцидента']
    group = row['Группа тем']
    words_in_text = re.findall(r'\w+', text.lower())
    for word in words_in_text:
        if word in word_group_mapping:
            word_group_mapping[word].add(group)
        else:
            word_group_mapping[word] = {group}
word_group_df = pd.DataFrame(word_group_mapping.items(), columns=['Слово', 'Группы тем'])

# Сохранение DataFrame в CSV
word_group_df.to_csv('word2topic_group.csv', index=False)

In [69]:
word_group_df

,Слово,Группы тем
0,добрый,"{памятники и объекты культурного наследия, соц..."
1,день,"{памятники и объекты культурного наследия, соц..."
2,сегодня,"{памятники и объекты культурного наследия, соц..."
3,20,"{здравоохранение/медицина, строительство и арх..."
4,08,"{здравоохранение/медицина, социальное обслужив..."
...,...,...
61645,боя,{образование}
61646,дошколят,{образование}
61647,разминка,{образование}
61648,12кв,{социальное обслуживание и защита}


In [70]:
word_group_mapping = {}

# Проход по каждой строке датасета
for index, row in df.iterrows():
    text = row['Исполнитель']
    group = row['Тема']
    words_in_text = re.findall(r'\w+', text.lower())
    for word in words_in_text:
        if word in word_group_mapping:
            word_group_mapping[word].add(group)
        else:
            word_group_mapping[word] = {group}
word_group_df = pd.DataFrame(word_group_mapping.items(), columns=['Исполнитель', 'тема'])

# Сохранение DataFrame в CSV
word_group_df.to_csv('executor2topic.csv', index=False)

In [71]:
word_group_df

,Исполнитель,тема
0,лысьвенский,"{региональное имущество, плата за вывоз тко, ★..."
1,городской,"{региональное имущество, плата за вывоз тко, ★..."
2,округ,"{региональное имущество, плата за вывоз тко, ★..."
3,министерство,"{плата за вывоз тко, региональное имущество, о..."
4,социального,"{плата за вывоз тко, региональное имущество, д..."
5,развития,"{плата за вывоз тко, региональное имущество, д..."
6,пк,"{плата за вывоз тко, региональное имущество, ★..."
7,город,"{плата за вывоз тко, региональное имущество, ★..."
8,пермь,"{плата за вывоз тко, региональное имущество, ★..."
9,здравоохранения,"{региональное имущество, отсутствие горячей во..."


In [122]:
word_in_topics(df,'труп')

['здравоохранение/медицина' 'погребение и похоронное дело']
------------------------------------------------------------------------------------------------------
Index([1382, 4285, 14954, 20036], dtype='int64')
'у нас в красновишерске , не давно был губернатор , только местные власти не сказали , ему как мы " хорошо " живём : " женщин рожать за 100 км. возят в соликамск , зубные кабинеты закрыли , со всеми серьёзными болезнями и да же трупы потрошить , надо опять ехать в соликамск !
'у нас в кудымкаре 5 ритуальных компаний все платят налоги за эту деятельность и стараются выживать на этом рынке. но есть среди этих компаний один монопольщик который находится около морга и морг тесно работает только с ним одним потому, что он платит моргу сумму за трупы. но ведь мы тоже хотим работать и кушать!!! просим вас как то повлиять на сотрудников морга, запретить брать плату за трупы и организовать поочередность среди ритуальных компаний.
'это бесполезно! сидит хамка и орет в трубку вызывайте уч

array(['здравоохранение/медицина', 'погребение и похоронное дело'],
      dtype=object)

In [126]:
trigger_words = ['убийство', 'смерть', 'убийца','боевые действия','терроризм',
                 'авария', 'гибель','наркотик','теракт','землетрясение','лесной пожар',
                 'критический','взрывное устройство','труп',
                 'взрывчатое устройство','террористическая группировка',
                 'заложник','нарушение пожарной безопасности','взрыв', 'бомба',
                 'оружие','общественная опасность','химическая авария',
                 'cмертельное ранение','грабители' ]


In [127]:
import csv

In [128]:
with open('trigger_words.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for word in trigger_words:
        csv_writer.writerow([word])

In [14]:
pip install emoji

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 933.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import emoji
import re

pattern_mail = re.compile(
    r"([a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)&(\d{11,12})"
)
pattern_link = re.compile(
    r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
)
pattern_number = re.compile(
    "\\+?\\d{1,4}?[-.\\s]?\\(?\\d{1,3}?\\)?[-.\\s]?\\d{1,4}[-.\\s]?\\d{1,4}[-.\\s]?\\d{1,9}"
)
pattern_digit = re.compile(r"\+?\d+")

pattern_data = re.compile(
    r"\b(\d{1,2}\W\d{1,2}\W\d{2,4}|\d{2,4}\W\d{1,2}\W\d{1,2})\b"
)



def preprocess_data(data: pd.DataFrame, demojize: bool = True) -> pd.DataFrame:
    """
    Preprocess the data

    Parameters
    ----------
    data : pd.DataFrame
        Input data to preprocess.
    demojize : bool
        Convert emojis to plain text (takes time!)
    """

    processed_data = data.copy()

    # Delete "'" at the start of the line.
    # data["Текст инцидента"].apply(lambda x : x[1:] if x[0] == "'" else x)
    processed_data["Текст инцидента"] = processed_data["Текст инцидента"].apply(
        lambda x: x[1:]
    )

    # Delete "*" from themes names.
    processed_data["Тема"] = processed_data["Тема"].apply(
        lambda x: x[2:] if x[:2] == "★ " else x
    )

    # Demojinize.
    if demojize:
        processed_data["Текст инцидента"] = processed_data[
            "Текст инцидента"
        ].apply(emoji.demojize)

    # Parse VK replies.
    VK_regexp = re.compile(r"\[(?P<ID>\w+)\|(?P<NAME>[^\[^\]]*)\]")
    # matches = VK_regexp.finditer(line)
    processed_data["Текст инцидента"] = processed_data["Текст инцидента"].apply(
        lambda x: VK_regexp.sub("\g<NAME>", x)
    )

    # Replace whitespace expressions.
    whitespace_substrings = {"<br>", "\\n"}
    whitespace_regexp = re.compile(
        "|".join(re.escape(substring) for substring in whitespace_substrings)
    )
    processed_data["Текст инцидента"] = processed_data[
        "Текст инцидента"
    ].str.replace(whitespace_regexp, " ", regex=True)
    # for substring in whitespace_substrings:
    #    processed_data["Текст инцидента"].str.replace(substring, " ")

    # Replace multiple whitespaces with one.
    multiple_whitespace_regexp = re.compile("\s+")
    processed_data["Текст инцидента"] = processed_data[
        "Текст инцидента"
    ].str.replace(multiple_whitespace_regexp, " ", regex=True)

    processed_data["Текст инцидента"] = processed_data[
        "Текст инцидента"
    ].str.replace(pattern_mail, "(почта)", regex=True)

    processed_data["Текст инцидента"] = processed_data[
        "Текст инцидента"
    ].str.replace(pattern_link, "(ccылка)", regex=True)

    processed_data["Текст инцидента"] = processed_data[
        "Текст инцидента"
    ].str.replace(pattern_data, "(дата)", regex=True)

    processed_data["Текст инцидента"] = processed_data[
        "Текст инцидента"
    ].str.replace(pattern_number, "(номер)", regex=True)

    return processed_data

In [16]:
DATA_FOLDER = '../data/raw/train_dataset_train.csv'
df = pd.read_csv(DATA_FOLDER, sep=';')

In [17]:
df_test = preprocess_data(df)

In [18]:
df_test.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, (дата), моя мать шла по ...",Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, (номер). В Перми с ноября 2021 года н...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день ! Скажите пожалуйста если подовала...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса з...,Технические проблемы с записью на прием к врачу


In [24]:
pattern_mail = re.compile(
    r"([a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)&(\d{11,12})"
)
pattern_link = re.compile(
    r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
)
pattern_number = re.compile(
    "\\+?\\d{1,4}?[-.\\s]?\\(?\\d{1,3}?\\)?[-.\\s]?\\d{1,4}[-.\\s]?\\d{1,4}[-.\\s]?\\d{1,9}"
)
pattern_digit = re.compile(r"\+?\d+")

pattern_data = re.compile(
    r"\b(\d{1,2}\W\d{1,2}\W\d{2,4}|\d{2,4}\W\d{1,2}\W\d{1,2})\b"
)

In [34]:
df_test['Текст инцидента'].apply(lambda x: re.match(pattern_digit, x)).unique()

array([None, <re.Match object; span=(0, 1), match='1'>,
       <re.Match object; span=(0, 2), match='14'>,
       <re.Match object; span=(0, 2), match='30'>,
       <re.Match object; span=(0, 2), match='16'>,
       <re.Match object; span=(0, 2), match='62'>,
       <re.Match object; span=(0, 1), match='1'>,
       <re.Match object; span=(0, 1), match='2'>,
       <re.Match object; span=(0, 1), match='2'>,
       <re.Match object; span=(0, 1), match='8'>,
       <re.Match object; span=(0, 2), match='32'>,
       <re.Match object; span=(0, 2), match='16'>,
       <re.Match object; span=(0, 2), match='22'>,
       <re.Match object; span=(0, 2), match='59'>,
       <re.Match object; span=(0, 2), match='18'>,
       <re.Match object; span=(0, 1), match='1'>,
       <re.Match object; span=(0, 2), match='22'>,
       <re.Match object; span=(0, 1), match='7'>,
       <re.Match object; span=(0, 2), match='68'>,
       <re.Match object; span=(0, 2), match='40'>,
       <re.Match object; span=(0

In [20]:
df_test.iloc[0,2]

'Добрый день. Сегодня, (дата), моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна. Уважаемая Администрация, сделайте с этим что-нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо!'

In [ ]:
for 